# Time-optimal path planning
This notebook handles standard code for the computation of time-optimal paths in real windfields

In [1]:
import os
import sys
import time
import numpy as np
from math import atan, cos, sin
sys.path.extend(['/home/bastien/Documents/work/mermoz', 
                 '/home/bastien/Documents/work/mermoz/src',
                 '/home/bastien/Documents/work/mdisplay',
                 '/home/bastien/Documents/work/mdisplay/src'])

from mermoz.feedback import TargetFB
from mermoz.mdf_manager import MDFmanager
from mermoz.params_summary import ParamsSummary
from mermoz.problem import MermozProblem
from mermoz.model import ZermeloGeneralModel
from mermoz.solver import Solver
from mermoz.stoppingcond import TimedSC, DistanceSC
from mermoz.wind import DiscreteWind
from mermoz.misc import *
from mdisplay.geodata import GeoData

from solver_utils import select_wind

### Problem definition

In [1]:
dd = select_wind()

NameError: name 'select_wind' is not defined

Initial point, target, UAV airspeed

In [2]:
gd = GeoData()
# Initial point
x_init = np.array(gd.get_coords('Dakar', units='rad'))

# Target
x_target = np.array(gd.get_coords('Natal', units='rad'))

# UAV airspeed in m/s
v_a = 23.

case_name = 'dakar-natal'

Set up the problem

In [4]:
# Prepare output directory
output_dir = f'/home/bastien/Documents/work/mermoz/output/example_solver_{case_name}'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Create a file manager to dump problem data
mdfm = MDFmanager()
mdfm.set_output_dir(output_dir)

# Problem coordinates
# coords = COORD_GCS
coords = COORD_GCS

# Problem wind data
total_wind = DiscreteWind()
total_wind.load(f'/home/bastien/Documents/data/wind/windy/{dd.value}.mz/data.h5')

# Time window upper bound
# Estimated through great circle distance + 20 percent
T = 30. * 3600 # 1.2 * geodesic_distance(x_init, x_target, mode='rad') / v_a

# Creates the cinematic model
zermelo_model = ZermeloGeneralModel(v_a, coords=coords)
zermelo_model.update_wind(total_wind)

# Creates the navigation problem on top of the previous model
mp = MermozProblem(zermelo_model, T=T, coords=coords, autodomain=True)

### Solver parameters

In [89]:
auto_psi = atan((cos(x_target[1]) * sin(x_target[0] - x_init[0])) / (
            cos(x_init[1]) * sin(x_target[1]) - sin(x_init[1]) * cos(x_target[1]) * cos(x_target[0] - x_init[0])))
auto_psi += pi
psi_min = auto_psi - DEG_TO_RAD * 20.
psi_max = auto_psi + DEG_TO_RAD * 20.

nt_pmp = 1000
opti_ceil = 0.5 * DEG_TO_RAD * EARTH_RADIUS
neighb_ceil = opti_ceil/2.

solver = Solver(mp,
                x_init,
                x_target,
                T,
                psi_min,
                psi_max,
                output_dir,
                N_disc_init=2,
                opti_ceil=opti_ceil,
                neighb_ceil=neighb_ceil,
                n_min_opti=1,
                adaptive_int_step=False,
                N_iter=nt_pmp)

solver.log_config()
solver.setup()

Solve problem

In [90]:
t_start = time.time()
solver.solve_fancy()
t_end = time.time()
time_pmp = t_end - t_start

Shooting 201.9
Shooting 241.9
Iteration 1 - Shooting 221.9 - Depth 1 - Optimum found
    * Pre-processing :  0.196 s
    * Processing      :  0.111 s
    * Total           :  0.307 s
107460.0


### Explicit control laws

In [91]:
# Stay-on-geodesic heuristic
# sc = DistanceSC(lambda x: geodesic_distance(x, x_target), opti_ceil)
sc = TimedSC(T)
mp.load_feedback(TargetFB(mp.model.wind, v_a, x_target, mp.coords))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

Dump everything to output directory

In [92]:
mdfm.dump_wind(total_wind)
mdfm.dump_trajs(mp.trajs)

ps = ParamsSummary({}, output_dir)
ps.load_from_solver(solver)
ps.add_param('nt_pmp', nt_pmp)
ps.add_param('pmp_time', time_pmp)
ps.dump()